# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import nltk 
nltk.download(['punkt','wordnet','averaged_perceptron_tagger'])
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# Install additional Python packages that are not pre-installed.
!pip install textblob  # For sentiment analysis and text processing.
!pip install dask  # For parallel computing and efficiency with large datasets.

    100% |████████████████████████████████| 645kB 15.1MB/s ta 0:00:01


In [3]:
import pandas as pd
import pickle # For serializing and deserializing Python object structures.
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC # SVM model for classification tasks.
from sklearn.multiclass import OneVsRestClassifier # Strategy for multi-class classification.
from sklearn.ensemble import RandomForestClassifier # For ensemble learning methods.
from textblob import TextBlob # For processing textual data.
import dask # For parallel computing.

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterRelief_R.db')
df = pd.read_sql_table('DisasterReliefMessage', engine)
X = df['message']
y = df.iloc[:,4:]
y['related'].replace(2, 1, inplace=True)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    Tokenizes text into words.

    Parameters:
    - text (str): The text to be tokenized.

    Returns:
    - list[str]: A list of words extracted from the text.
    """
     
    text = re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    tokens=word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens=[]
    for word in tokens:
        clean_word = lemmatizer.lemmatize(word).lower().strip()
        clean_tokens.append(clean_word)
    return clean_tokens
        

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)

for i in range(len(y_test.columns)):
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

             precision    recall  f1-score   support

          0       0.63      0.37      0.46      1791
          1       0.83      0.94      0.88      6063

avg / total       0.79      0.81      0.79      7854

             precision    recall  f1-score   support

          0       0.89      0.99      0.94      6546
          1       0.84      0.39      0.53      1308

avg / total       0.88      0.89      0.87      7854

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7816
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      7854

             precision    recall  f1-score   support

          0       0.73      0.87      0.80      4620
          1       0.75      0.55      0.63      3234

avg / total       0.74      0.74      0.73      7854

             precision    recall  f1-score   support

          0       0.93      0.99      0.96      7286
          1       0.58      0.09 

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# Define a Dask delayed function to wrap the GridSearchCV
@dask.delayed
def grid_search_cv(X_train, y_train):
    """
    Performs grid search cross-validation on a given model with specified parameters.

    Parameters:
    - parameters (dict): The parameters to tune.
    - model (Model): The model to be tuned.
    - cv_data (Data): The data used for cross-validation.

    Returns:
    - GridSearchCV object: The result of grid search cross-validation.
    """
    # Define the pipeline with CountVectorizer, TF-IDF, and Logistic Regression
    pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state = 0))))
        ])

    # Define parameters for grid search
    parameters = {
                  'tfidf__smooth_idf':[True, False],
                  'clf__estimator__estimator__C': [1, 2, 5]
                 }

    # Grid Search
    cv = GridSearchCV(pipeline, param_grid=parameters, scoring='precision_samples', cv=3, n_jobs=-1, verbose=2)
    cv.fit(X_train, y_train)
    
    print("Accuracy")
    print((y_pred == y_test).mean())

    for i, column_name in enumerate(y_test.columns):
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    
    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
# Call the delayed function
result = grid_search_cv(X_train, y_train)

# Compute the result with Dask using the threaded scheduler
result.compute(scheduler='threads')

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=True ..........
[CV]  clf__estimator__estimator__C=1, tfidf__smooth_idf=True, total=   8.6s
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=True ..........


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   13.2s remaining:    0.0s


[CV]  clf__estimator__estimator__C=1, tfidf__smooth_idf=True, total=   8.7s
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=True ..........
[CV]  clf__estimator__estimator__C=1, tfidf__smooth_idf=True, total=   8.5s
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=False .........
[CV]  clf__estimator__estimator__C=1, tfidf__smooth_idf=False, total=   8.5s
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=False .........
[CV]  clf__estimator__estimator__C=1, tfidf__smooth_idf=False, total=   8.4s
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=False .........
[CV]  clf__estimator__estimator__C=1, tfidf__smooth_idf=False, total=   8.7s
[CV] clf__estimator__estimator__C=2, tfidf__smooth_idf=True ..........
[CV]  clf__estimator__estimator__C=2, tfidf__smooth_idf=True, total=   9.3s
[CV] clf__estimator__estimator__C=2, tfidf__smooth_idf=True ..........
[CV]  clf__estimator__estimator__C=2, tfidf__smooth_idf=True, total=   9.1s
[CV] clf__estimator__estimator__C=2, tf

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  4.3min finished


Accuracy
related                   0.806086
request                   0.886300
offer                     0.995162
aid_related               0.736822
medical_help              0.929463
medical_products          0.955309
search_and_rescue         0.974663
security                  0.984085
military                  0.968424
child_alone               1.000000
water                     0.943850
food                      0.922842
shelter                   0.926152
clothing                  0.984085
money                     0.977591
missing_people            0.988414
refugees                  0.970588
death                     0.962185
other_aid                 0.869493
infrastructure_related    0.935829
transport                 0.959129
buildings                 0.951617
electricity               0.978228
tools                     0.992615
hospitals                 0.989177
shops                     0.995671
aid_centers               0.989814
other_infrastructure      0.955437
weather_rel

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...nalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__smooth_idf': [True, False], 'clf__estimator__estimator__C': [1, 2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='precision_samples', verbose=2)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
# Custom transformer to extract sentiment scores
class SentimentExtractor(BaseEstimator, TransformerMixin):
    """
    A custom transformer that extracts sentiment from text and adds it as a feature to the dataset.

    Inherits from:
    - BaseEstimator: Provides methods for getting and setting parameters.
    - TransformerMixin: Provides a default implementation of the fit_transform method.

    Parameters:
    - sentiment_model: A sentiment analysis model used to compute sentiment scores.

    Attributes:
    - sentiment_features_: A list of feature names created by the transformer. This attribute is generated
      during fitting and used in transforming data to ensure consistency in output format.

    Methods:
    - fit: Fits the extractor to the data.
    - transform: Transforms data by extracting sentiments.
    """
    
    def fit(self, X, y=None):
        """
        Fits the sentiment extractor to the data.

        Parameters:
        - X (DataFrame): The input data to fit.
        - y (ignored): Placeholder for compatibility with scikit-learn.

        Returns:
        - self: The instance itself.
        """
        return self
    
    def transform(self, X):
        """
        Transforms the input data by extracting sentiment scores.

        Parameters:
        - X (DataFrame): The data to be transformed.

        Returns:
        - DataFrame: The transformed data with added sentiment features.
        """
        return [[TextBlob(text).sentiment.polarity] for text in X]

# Define a Dask delayed function to wrap the GridSearchCV
@dask.delayed
def grid_search_cv_mod(X_train, y_train):
    """
    An modified version of grid search cross-validation for additional customization.

    Parameters:
    - X training data
    - y training data

    Returns:
    - GridSearchCV object: The result of grid search cross-validation with modifications.
    """

    # Define the pipeline with CountVectorizer, TF-IDF, and RandomForestClassifier
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('sentiment', SentimentExtractor())
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    # Define parameters for grid search
    parameters = {
        'features__text__tfidf__use_idf': (True, False),
        'clf__estimator__max_depth': [None, 10]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters, scoring='precision_samples', cv=3, n_jobs=-1, verbose=2)
    cv.fit(X_train, y_train)
    
    y_pred = cv.predict(X_test)

    for i in range(len(y_test.columns)):
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    return cv


In [12]:
# Call the delayed function of modified
result_mod = grid_search_cv_mod(X_train, y_train)

# Compute the result with Dask using the threaded scheduler
result_mod.compute(scheduler='threads')

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__max_depth=None, features__text__tfidf__use_idf=True 
[CV]  clf__estimator__max_depth=None, features__text__tfidf__use_idf=True, total=  41.8s
[CV] clf__estimator__max_depth=None, features__text__tfidf__use_idf=True 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   55.2s remaining:    0.0s


[CV]  clf__estimator__max_depth=None, features__text__tfidf__use_idf=True, total=  41.2s
[CV] clf__estimator__max_depth=None, features__text__tfidf__use_idf=True 
[CV]  clf__estimator__max_depth=None, features__text__tfidf__use_idf=True, total=  42.2s
[CV] clf__estimator__max_depth=None, features__text__tfidf__use_idf=False 
[CV]  clf__estimator__max_depth=None, features__text__tfidf__use_idf=False, total=  41.7s
[CV] clf__estimator__max_depth=None, features__text__tfidf__use_idf=False 
[CV]  clf__estimator__max_depth=None, features__text__tfidf__use_idf=False, total=  41.3s
[CV] clf__estimator__max_depth=None, features__text__tfidf__use_idf=False 
[CV]  clf__estimator__max_depth=None, features__text__tfidf__use_idf=False, total=  41.7s
[CV] clf__estimator__max_depth=10, features__text__tfidf__use_idf=True 
[CV]  clf__estimator__max_depth=10, features__text__tfidf__use_idf=True, total=  19.5s
[CV] clf__estimator__max_depth=10, features__text__tfidf__use_idf=True 
[CV]  clf__estimator__

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  8.6min finished


             precision    recall  f1-score   support

          0       1.00      0.00      0.01      1791
          1       0.77      1.00      0.87      6063

avg / total       0.82      0.77      0.67      7854

             precision    recall  f1-score   support

          0       0.83      1.00      0.91      6546
          1       1.00      0.00      0.00      1308

avg / total       0.86      0.83      0.76      7854

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7816
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      7854

             precision    recall  f1-score   support

          0       0.63      0.98      0.77      4620
          1       0.89      0.17      0.28      3234

avg / total       0.73      0.65      0.57      7854

             precision    recall  f1-score   support

          0       0.93      1.00      0.96      7286
          1       1.00      0.01 

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, ma...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'features__text__tfidf__use_idf': (True, False), 'clf__estimator__max_depth': [None, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='precision_samples', verbose=2)

### 9. Export your model as a pickle file

In [13]:
pickle_file_path = 'model.pkl'
pickle.dump(pipeline, open(pickle_file_path, 'wb')) #dumping pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.